# Regressão em Dados de Painel


Neste notebook apresentamos conceitos de regressão em dados de painel usando um caso prático de análise de dados reais. A análise descrita usa dados sobre a pandemia de Covid-19 em algumas das maiores cidades brasileiras, incluindo mobilidade, políticas públicas e casos de Covid. Os dados foram obtidos das seguintes fontes:

- Mobilidade: [Google Mobility Reports](https://www.google.com/covid19/mobility/)

- Políticas públicas: [Brazilian Sub-National Covid-19 Policy Responses](https://github.com/OxCGRT/Brazil-covid-policy)

- Casos e mortes: [Brasil.io](https://brasil.io/)


A análise é baseada nas bibliotecas [StatsModels](https://www.statsmodels.org) e [linearmodels](https://bashtage.github.io/linearmodels/index.html). A biblioteca `linearmodels` tem funcionalidades específicas para regressão em dados de painel.

O objetivo da análise é entender como o índice de rigidez das políticas de uma cidade (StringencyIndex) e o número de mortes afeta a mobilidade das pessoas.

In [47]:
# Importação de pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import statsmodels.api as sm
import statsmodels.formula.api as smf
import linearmodels as lm

## Lendo dados de um arquivo

Os dados coletados foram agregados em um arquivo CSV, exibido abaixo e disponibilizado no reposítório deste tutorial.

In [48]:
# lê o arquivo CSV
df = pd.read_csv('../../data/covid_politicas_mobilidade.csv', parse_dates=['Date'])
# mostra o conteúdo do DataFrame
df.head(3)

,State,City,Date,StringencyIndex,city_ibge_code,new_confirmed,new_deaths,confirmed,deaths,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,avg_mobility
0,para,ananindeua,2020-03-25,80.56,1500800,2,0,6,0,-51.0,-17.0,-59.0,-70.0,-40.0,17.0,-47.4
1,para,ananindeua,2020-03-26,80.56,1500800,2,0,8,0,-50.0,-14.0,-54.0,-69.0,-39.0,17.0,-45.2
2,para,ananindeua,2020-03-27,83.33,1500800,0,0,10,0,-51.0,-11.0,-58.0,-71.0,-36.0,18.0,-45.4


## Pooled OLS

O primeiro modelo construído representa uma regressão usual, considerando cada linha do nosso dataframe como uma observação. Os resultados são apresentados abaixo. Percebe-se que tanto a rigidez das políticas quanto o número de mortes ajudam a diminuir a mobilidade das pessoas.

Este tipo de regressão simples não é indicado para estes casos. A razão principal é que o modelo não considera fatores não observados que podem estar associados à dinâmica estudada. Por exemplo, cidades com níveis econômicos mais baixos podem ter menos capacidade para a prática de distanciamento social.

In [49]:
model = smf.ols("avg_mobility ~ StringencyIndex  + new_deaths", data=df)
response = model.fit()
response.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           avg_mobility   R-squared:                       0.282
Model:                            OLS   Adj. R-squared:                  0.282
Method:                 Least Squares   F-statistic:                     4880.
Date:                Mon, 17 Oct 2022   Prob (F-statistic):               0.00
Time:                        08:39:24   Log-Likelihood:            -1.0685e+05
No. Observations:               24851   AIC:                         2.137e+05
Df Residuals:                   24848   BIC:                         2.137e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          41.3338      0.587     70.412      0.000      40.183      42.484
StringencyIndex    -0.8277      0.009    -96.712      0.000      -0.844      -0.811
new_deaths         -0.1399      0.006    -25.091      0.000      -0.151      -0.129
==============================================================================
Omnibus:                       97.946   Durbin-Watson:                   0.368
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              133.600
Skew:                           0.019   Prob(JB):                     9.75e-30
Kurtosis:                       3.357   Cond. No.                         356.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Efeitos Fixos

Uma forma de se considerar estes fatores não observados é usar efeitos fixos (fixed effects). Para isto, adocionamos variáveis que representam todos os efeitos relacionados com nossas entidades que podem estar afetando o modelo. 

### Variáveis dummies

Uma forma simples de se fazer isto é adicionar dummies das nossas entidades (no caso, cidades). O exemplo abaixo mostra isso. Os coeficientes das cidades no modelo representam todos os fatores específicos de cada cidade que podem afetar o nível de mobilidade, como questões econômicas ou a matriz de transporte.

In [50]:
model = smf.ols("avg_mobility ~ StringencyIndex  + new_deaths + C(City)", data=df)
response = model.fit()
response.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           avg_mobility   R-squared:                       0.559
Model:                            OLS   Adj. R-squared:                  0.558
Method:                 Least Squares   F-statistic:                     581.2
Date:                Mon, 17 Oct 2022   Prob (F-statistic):               0.00
Time:                        08:39:25   Log-Likelihood:            -1.0081e+05
No. Observations:               24851   AIC:                         2.017e+05
Df Residuals:                   24796   BIC:                         2.022e+05
Df Model:                          54                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                             66.6522      0.806     82.731      0.000      65.073      68.231
C(City)[T.aparecida de goiania]      -10.8433      0.905    -11.984      0.000     -12.617      -9.070
C(City)[T.aracaju]                   -28.2396      0.904    -31.255      0.000     -30.011     -26.469
C(City)[T.araguaina]                  -8.2407      0.911     -9.050      0.000     -10.026      -6.456
C(City)[T.arapiraca]                 -13.5524      0.920    -14.730      0.000     -15.356     -11.749
C(City)[T.belem]                     -12.5698      0.906    -13.867      0.000     -14.346     -10.793
C(City)[T.belo horizonte]            -17.7445      0.906    -19.581      0.000     -19.521     -15.968
C(City)[T.boa vista]                  -3.0422      0.908     -3.349      0.001      -4.822      -1.262
C(City)[T.campina grande]            -13.4059      0.909    -14.742      0.000     -15.188     -11.624
C(City)[T.campo grande]              -16.1572      0.903    -17.884      0.000     -17.928     -14.386
C(City)[T.caucaia]                    -6.5758      0.910     -7.225      0.000      -8.360      -4.792
C(City)[T.caxias do sul]             -13.7376      0.902    -15.223      0.000     -15.506     -11.969
C(City)[T.cruzeiro do sul]            -3.3742      0.936     -3.603      0.000      -5.210      -1.539
C(City)[T.cuiaba]                    -24.5812      0.906    -27.117      0.000     -26.358     -22.804
C(City)[T.curitiba]                  -18.9520      0.905    -20.951      0.000     -20.725     -17.179
C(City)[T.dourados]                  -21.8885      0.910    -24.065      0.000     -23.671     -20.106
C(City)[T.feira de santana]          -14.6933      0.899    -16.339      0.000     -16.456     -12.931
C(City)[T.florianopolis]             -39.0875      1.044    -37.444      0.000     -41.134     -37.041
C(City)[T.fortaleza]                 -23.2648      0.909    -25.582      0.000     -25.047     -21.482
C(City)[T.goiania]                   -18.8191      0.904    -20.822      0.000     -20.591     -17.048
C(City)[T.guarulhos]                 -21.5173      0.907    -23.729      0.000     -23.295     -19.740
C(City)[T.imperatriz]                 -0.9062      0.914     -0.992      0.321      -2.697       0.884
C(City)[T.jaboatao dos guararapes]    -8.3031      0.904     -9.181      0.000     -10.076      -6.531
C(City)[T.ji-parana]                   7.4673      0.915      8.160      0.000       5.674       9.261
C(City)[T.joao pessoa]               -29.4349      0.905    -32.523      0.000     -31.209     -27.661
C(City)[T.joinville]                 -11.1101      0.903    -12.309      0.000     -12.879      -9.341
C(City)[T.lagarto]                    -7.1643      0.929     -7.711      0.000      -8.985      

Também podemos pensar em efeitos que são especícos para os períodos de tempo. Por exemplo, é rezoável imaginar que no início da pandemia as pessoas reagiam com mais intensidade às notícias de número de mortes e com isso tinham maior tendência a se distanciar. Para capturar estes efeitos que são fixos nos períodos de tempo podemos também adicionar dummies para a variável de tempo:

In [51]:
model = smf.ols("avg_mobility ~ StringencyIndex  + new_deaths + C(City) + C(Date)", data=df)
response = model.fit()
response.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           avg_mobility   R-squared:                       0.837
Model:                            OLS   Adj. R-squared:                  0.833
Method:                 Least Squares   F-statistic:                     223.9
Date:                Mon, 17 Oct 2022   Prob (F-statistic):               0.00
Time:                        08:39:40   Log-Likelihood:                -88430.
No. Observations:               24851   AIC:                         1.780e+05
Df Residuals:                   24293   BIC:                         1.825e+05
Df Model:                         557                                         
Covariance Type:            nonrobust                                         
===============================================================================================================
                                                  coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------
Intercept                                      -6.3696      8.615     -0.739      0.460     -23.255      10.516
C(City)[T.aparecida de goiania]                -9.2825      0.556    -16.704      0.000     -10.372      -8.193
C(City)[T.aracaju]                            -27.9878      0.555    -50.437      0.000     -29.075     -26.900
C(City)[T.araguaina]                          -10.0349      0.559    -17.943      0.000     -11.131      -8.939
C(City)[T.arapiraca]                          -15.8110      0.565    -27.975      0.000     -16.919     -14.703
C(City)[T.belem]                               -9.0502      0.557    -16.244      0.000     -10.142      -7.958
C(City)[T.belo horizonte]                     -21.1440      0.558    -37.917      0.000     -22.237     -20.051
C(City)[T.boa vista]                           -9.1477      0.560    -16.339      0.000     -10.245      -8.050
C(City)[T.campina grande]                     -10.8941      0.559    -19.499      0.000     -11.989      -9.799
C(City)[T.campo grande]                       -14.8159      0.555    -26.699      0.000     -15.904     -13.728
C(City)[T.caucaia]                            -10.3037      0.560    -18.416      0.000     -11.400      -9.207
C(City)[T.caxias do sul]                      -16.5418      0.555    -29.811      0.000     -17.629     -15.454
C(City)[T.cruzeiro do sul]                    -15.1708      0.582    -26.071      0.000     -16.311     -14.030
C(City)[T.cuiaba]                             -21.2102      0.557    -38.073      0.000     -22.302     -20.118
C(City)[T.curitiba]                           -21.9794      0.557    -39.477      0.000     -23.071     -20.888
C(City)[T.dourados]                           -20.9249      0.559    -37.464      0.000     -22.020     -19.830
C(City)[T.feira de santana]                   -14.4094      0.553    -26.067      0.000     -15.493     -13.326
C(City)[T.florianopolis]                      -44.6856      0.645    -69.326      0.000     -45.949     -43.422
C(City)[T.fortaleza]                          -25.2844      0.560    -45.189      0.000     -26.381     -24.188
C(City)[T.goiania]                            -16.0890      0.555    -28.968      0.000     -17.178     -15.000
C(City)[T.guarulhos]                          -15.3887      0.559    -27.553      0.000     -16.483     -14.294
C(City)[T.imperatriz]                           5.0596      0.563      8.987      0.000       3.956       6.163
C(City)[T.jaboatao dos guararapes]             -8.5503      0.555    -15.396      0.000      -9.639      -7.462
C(City)[T.ji-parana]                           -3.1921      0.568     -5.619      0.000      -4.306      -2.079
C(City)[T.joao pessoa]                        -28.2963     

Esta abordagem de se adicionar dummies para os efeitos tem dois problemas: (i) adiciona muitas variáveis, aumentando muito o grau de liberdade do modelo, fazendo com que fique menos confiável; e (ii) pode criar problemas de endogeneidade, com o coeficiente da entidade fazendo com que o erro do modelo seja correlacionado com os preditores.

### Within transformation

Para tratar estes problemas, of modelos de fixed effects fazem transformações matemáticas que eliminam os termos das variáveis não observadas, criando modelos mais consistentes. A transformação mais comum é chamada de within transformation, e é usada na biblioteca `linearmodels` como mostram os exemplos abaixo.

In [52]:
# é preciso indicar as entities e o time no índice do dataframe
df_fe = df.set_index(["City", "Date"])

df_fe.head()

State  StringencyIndex  city_ibge_code  new_confirmed  \
City       Date                                                               
ananindeua 2020-03-25  para            80.56         1500800              2   
           2020-03-26  para            80.56         1500800              2   
           2020-03-27  para            83.33         1500800              0   
           2020-03-28  para            83.33         1500800              0   
           2020-03-29  para            83.33         1500800              0   

                       new_deaths  confirmed  deaths  \
City       Date                                        
ananindeua 2020-03-25           0          6       0   
           2020-03-26           0          8       0   
           2020-03-27           0         10       0   
           2020-03-28           0         13       0   
           2020-03-29           0         14       0   

                       retail_and_recreation_percent_change_from_baseline  \
City       Date                                                             
ananindeua 2020-03-25                                              -51.0    
           2020-03-26                                              -50.0    
           2020-03-27                                              -51.0    
           2020-03-28                                              -56.0    
           2020-03-29                                              -60.0    

                       grocery_and_pharmacy_percent_change_from_baseline  \
City       Date                                                            
ananindeua 2020-03-25                                              -17.0   
           2020-03-26                                              -14.0   
           2020-03-27                                              -11.0   
           2020-03-28                                              -16.0   
           2020-03-29                                              -18.0   

                       parks_percent_change_from_baseline  \
City       Date                                             
ananindeua 2020-03-25                               -59.0   
           2020-03-26                               -54.0   
           2020-03-27                               -58.0   
           2020-03-28                               -57.0   
           2020-03-29                               -56.0   

                       transit_stations_percent_change_from_baseline  \
City       Date                                                        
ananindeua 2020-03-25                                          -70.0   
           2020-03-26                                          -69.0   
           2020-03-27                                          -71.0   
           2020-03-28                                          -70.0   
           2020-03-29                                          -77.0   

                       workplaces_percent_change_from_baseline  \
City       Date                                                  
ananindeua 2020-03-25                                    -40.0   
           2020-03-26                                    -39.0   
           2020-03-27                                    -36.0   
           2020-03-28                                    -29.0   
           2020-03-29                                    -20.0   

                       residential_percent_change_from_baseline  avg_mobility  
City       Date                                                                
ananindeua 2020-03-25                                      17.0         -47.4  
           2020-03-26                                      17.0         -45.2  
           2020-03-27                                      18.0         -45.4  
           2020-03-28                                      15.0         -45.6  
           2020-03-29                                      14.0         -46.2

O código abaixo aplica o fixed effects para entidades:

In [53]:
exog_vars = ["StringencyIndex", "new_deaths"]
exog = sm.add_constant(df_fe[exog_vars])
endog = df_fe["avg_mobility"]
mod_entity = lm.PanelOLS(endog, exog, entity_effects=True, time_effects=False)
res_entity = mod_entity.fit()
print(fe_res)

/home/luizcelso/anaconda3/lib/python3.7/site-packages/linearmodels/shared/exceptions.py:37: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:           avg_mobility   R-squared:                        0.0504
Estimator:                   PanelOLS   R-squared (Between):              0.0342
No. Observations:               24851   R-squared (Within):               0.1672
Date:                Thu, Oct 13 2022   R-squared (Overall):              0.1349
Time:                        10:53:54   Log-likelihood                -8.843e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      644.92
Entities:                          53   P-value                           0.0000
Avg Obs:                       468.89   Distribution:                 F(2,24293)
Min Obs:                       293.00                                           
Max Obs:                       504.00   F-statistic (robust):             644.92
                            

Podemos também incluir time effects:

In [54]:
exog_vars = ["StringencyIndex", "new_deaths"]
exog = sm.add_constant(df_fe[exog_vars])
endog = df_fe["avg_mobility"]
mod_entity_time = lm.PanelOLS(endog, exog, entity_effects=True, time_effects=True)
res_entity_time = mod_entity_time.fit()
print(res_entity_time)

                          PanelOLS Estimation Summary                           
Dep. Variable:           avg_mobility   R-squared:                        0.0504
Estimator:                   PanelOLS   R-squared (Between):              0.0342
No. Observations:               24851   R-squared (Within):               0.1672
Date:                Mon, Oct 17 2022   R-squared (Overall):              0.1349
Time:                        08:39:42   Log-likelihood                -8.843e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      644.92
Entities:                          53   P-value                           0.0000
Avg Obs:                       468.89   Distribution:                 F(2,24293)
Min Obs:                       293.00                                           
Max Obs:                       504.00   F-statistic (robust):             644.92
                            

O `linearmodels` tem uma função para se comparar modelos lado a lado. Abaixo podemos ver os dois modelos criados. No modelo com os dois efeitos, as duas variáveis são estatisticamente significativas.

In [55]:
from linearmodels.panel import compare

print(compare({"Entity effects": res_entity, "Entity and Time effects": res_entity_time},
             precision= 'std_errors', stars=True))

                        Model Comparison                        
                          Entity effects Entity and Time effects
----------------------------------------------------------------
Dep. Variable               avg_mobility            avg_mobility
Estimator                       PanelOLS                PanelOLS
No. Observations                   24851                   24851
Cov. Est.                     Unadjusted              Unadjusted
R-squared                         0.4205                  0.0504
R-Squared (Within)                0.4205                  0.1672
R-Squared (Between)              -0.2308                  0.0342
R-Squared (Overall)               0.2527                  0.1349
F-statistic                       8997.8                  644.92
P-value (F-stat)                  0.0000                  0.0000
=====================     ==============          ==============
const                          51.712***                  0.3058
                         